In [1]:
from fastai.collab import *
from fastai.tabular import *

## Collaborative filtering example with a truncated dataset


In [2]:
path = untar_data(URLs.ML_SAMPLE)
path

PosixPath('/home/stan_demontmarin/.fastai/data/movie_lens_sample')

In [3]:
ratings = pd.read_csv(path/'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,73,1097,4.0,1255504951
1,561,924,3.5,1172695223
2,157,260,3.5,1291598691
3,358,1210,5.0,957481884
4,130,316,2.0,1138999234


In [4]:
data = CollabDataBunch.from_df(ratings, seed=42)

In [5]:
y_range = [0,5.5]

In [6]:
learn = collab_learner(data, n_factors=50, y_range=y_range)

In [7]:
learn.fit_one_cycle(3, 5e-3)

epoch,train_loss,valid_loss,time
0,1.611772,0.911910,00:00
1,0.848646,0.676755,00:00
2,0.656772,0.671631,00:00


In [8]:
series2cat??